In [1]:
%load_ext tikzmagic
%matplotlib inline

# 스케일링과 다중공선성

지금까지 선형회귀분석을 할 때 항상 독립변수값의 평균을 0, 분산을 1로 만드는 스케일링(scaling)을 하도록 권장하였다. 이 절에서는 스케일링이 어떤 영향을 미치고 왜 필요한 것인지를 설명한다. 또한 독립변수간의 상관관계도 회귀분석에 어떤 영향을 미칠 수 있는지 공부한다.

## 스케일링

독립변수의 값은 사용하는 단위에 따라 아주 작은 값이 될 수도 있고 아주 큰 값이 될 수도 있다. 이러한 단위를 재설정하여 독립변수의 평균을 0, 분산을 1로 만드는 작업을 스케일링이라고 한다고 설명하였다. 실제로 보스턴 집값 데이터를 사용하여 스케일링의 영향을 살펴보자.

예제로 보스턴 집값 medv 를 두개의 독립변수 tax, nox 로 선형회귀분석을 해본다.

In [2]:
import statsmodels.api as sm

boston = sm.datasets.get_rdataset("Boston", "MASS").data[["nox", "tax", "medv"]]

우선 스케일링을 한 회귀분석 결과는 다음과 같다.

In [3]:
formula_scaled = "medv ~ scale(tax) + scale(nox)"
result_scaled = sm.OLS.from_formula(formula_scaled, boston).fit()
print(result_scaled.summary())

                            OLS Regression Results                            
Dep. Variable:                   medv   R-squared:                       0.243
Model:                            OLS   Adj. R-squared:                  0.240
Method:                 Least Squares   F-statistic:                     80.79
Date:                Fri, 12 Aug 2022   Prob (F-statistic):           3.75e-31
Time:                        15:44:44   Log-Likelihood:                -1769.8
No. Observations:                 506   AIC:                             3546.
Df Residuals:                     503   BIC:                             3558.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     22.5328      0.356     63.222      0.0

이번에는 스케일링을 하지 않는 결과를 살펴보자.

In [4]:
formula_notscaled = "medv ~ tax + nox"
result_notscaled = sm.OLS.from_formula(formula_notscaled, boston).fit()
print(result_notscaled.summary())

                            OLS Regression Results                            
Dep. Variable:                   medv   R-squared:                       0.243
Model:                            OLS   Adj. R-squared:                  0.240
Method:                 Least Squares   F-statistic:                     80.79
Date:                Fri, 12 Aug 2022   Prob (F-statistic):           3.75e-31
Time:                        15:44:44   Log-Likelihood:                -1769.8
No. Observations:                 506   AIC:                             3546.
Df Residuals:                     503   BIC:                             3558.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     38.9878      1.784     21.857      0.0

두 회귀분석 결과 요약을 비교하면 결정계수를 포함한 모든 성능 값이나 가중치의 유의확률등이 동일한 것을 알 수 있다. 물론 독립변수의 값이 다르므로 가중치 자체의 값은 달라지지만 스케일링 시에 독립변수 값이 커진(혹은 작아진)만큼 가중치가 작아(커)지므로 실질적으로는 차이가 없다. 실제로 두가지 결과 객체로 예측을 해보아도 예측값은 같아진다. 

실질적인 차이가 있는 부분은 우측 하단에 표시된 `Cond. No.` 라는 값과 새롭게 나타난 두번째 주의사항이다. 


<span class="caption">[스케일을 한 경우의 결과]</span>
```
                            OLS Regression Results                            
==============================================================================




                                        Cond. No.                         2.24
==============================================================================
```


<span class="caption">[스케일을 하지 않은 경우의 결과]</span>
```
                            OLS Regression Results                            
==============================================================================




                                        Cond. No.                     5.48e+03
==============================================================================
Notes:

[2] The condition number is large, 5.48e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
```

`Cond. No.` 라는 값은 <b>조건수(condition number)</b>라는 값이다. 조건수는 가중치를 구할 때 사용하는 다음 수식

$$
\hat{w} = (X^TX)^{-1}X^T y
$$

에서 $X^TX$ 행렬의 가장 작은 고유값(eigen-value) $\lambda_{\text{min}}$와 가장 큰 고유값 $\lambda_{\text{max}}$의 비율을 나타낸다.

$$
\text{조건수(condition number)} = \dfrac{\lambda_{\text{max}}}{\lambda_{\text{min}}}
$$

스케일링을 하지 않은 경우에 나타난 주의사항을 해석하면 다음과 같다.

```
[2] 조건수가 5,480으로 크다. 이는 강한 다중공선성이 있거나 수치적인 문제가 있다는 것을 의미할 수 있다. 
```


선형회귀분석에서 조건수는 선형회귀분석 결과의 <b>수치적 안정성(numerical stability)</b>을 나타내는 값이다. 수치적 안정성이라는 것은 우리가 선형회귀분석에 사용한 독립변수값에 오류가 있었을 경우 이 오류에 얼마나 민감하게 반응하는가를 나타낸다.

우리가 분석에 사용하는 독립변수의 값은 측정에 의한 것이므로 반올림 오차 등을 포함한 노이즈(noise) 오차를 항상 어느정도 가지게 된다. 조건수가 클수록 수치적 안정성이 떨어져 오차에 대해 더 민감하게 반응한다.

예시를 위해 우리가 가진 보스턴 집값 데이터에 일부러 어느정도 노이즈 오차를 추가해 보았다.

In [5]:
from numpy.random import seed, rand

boston_noise = boston.copy()
seed(0)
boston_noise["tax"] += boston_noise["tax"] * np.random.rand(len(boston)) * 0.01
boston_noise["nox"] += boston_noise["nox"] * np.random.rand(len(boston)) * 0.01

pd.concat([boston, boston_noise], axis=1, keys=["원본", "노이즈 오차 추가"])

원본            노이즈 오차 추가                  
       nox  tax  medv       nox         tax  medv
0    0.538  296  24.0  0.542639  297.624488  24.0
1    0.469  242  21.6  0.469228  243.730758  21.6
2    0.469  242  34.7  0.470190  243.458687  34.7
3    0.458  222  33.4  0.460043  223.209641  33.4
4    0.458  222  36.2  0.458479  222.940514  36.2
..     ...  ...   ...       ...         ...   ...
501  0.573  273  22.4  0.573176  274.018385  22.4
502  0.573  273  20.6  0.574286  274.433169  20.6
503  0.573  273  23.9  0.578465  275.049124  23.9
504  0.573  273  22.0  0.576337  273.910475  22.0
505  0.573  273  11.9  0.573616  275.522953  11.9

[506 rows x 6 columns]

이렇게 오차를 추가한 값으로 다시 선형회귀분석을 하여 가중치가 얼마나 달라지는지 살펴보자.

In [6]:
result_scaled_noise = sm.OLS.from_formula(formula_scaled, boston_noise).fit()
result_notscaled_noise = sm.OLS.from_formula(formula_notscaled, boston_noise).fit()

스케일링을 한 경우에는 tax, nox의 가중치가 각각 5%, -6% 정도 변화하였다.

In [7]:
error_scaled = (result_scaled.params - result_scaled_noise.params) / result_scaled.params * 100
pd.concat([result_scaled.params, result_scaled_noise.params, error_scaled], 
          axis=1, keys=["원본 데이터를 사용한 경우", "노이즈 오차가 있는 경우", "가중치 오류(%)"])

,원본 데이터를 사용한 경우,노이즈 오차가 있는 경우,가중치 오류(%)
Intercept,22.532806,22.532806,0.000000
scale(tax),-3.037683,-3.036102,0.052051
scale(nox),-1.896985,-1.898227,-0.065452


하지만 스케일링을 하지 않은 경우에는 tax, nox의 가중치가 각각 59%, 45% 정도 변화하였다.

In [8]:
error_notscaled = (result_notscaled.params - result_notscaled_noise.params) / result_notscaled.params * 100
pd.concat([result_notscaled.params, result_notscaled_noise.params, error_notscaled], 
          axis=1, keys=["원본 데이터를 사용한 경우", "노이즈 오차가 있는 경우", "가중치 오류(%)"])

,원본 데이터를 사용한 경우,노이즈 오차가 있는 경우,가중치 오류(%)
Intercept,38.987751,38.984622,0.008025
tax,-0.018042,-0.017935,0.590006
nox,-16.386784,-16.312490,0.453375


따라서 일반적인 경우에는 모든 독립변수에 대해 스케일링을 하는 것이 안전하고 권장된다.

## 다중공선성

다음으로 스케일링을 하지 않았을 때 나타난 주의사항에서 언급된 "다중공선성"이란 용어에 대해 설명한다. <b>다중공선성(multi-collinearity)</b>은 독립변수 사이에 강한 상관관계가 있음을 의미한다. 일반적으로 독립변수들은 상관관계가 강할수록 회귀분석에 나쁜 영향을 미친다. 최악의 경우는 상관계수가 1이거나 선형종속관계가 있는 경우다.

<b>선형종속(linearly-dependent)</b>이란 하나 이상의 독립변수를 선형조합(linear-combination)하면 다른 독립변수와 같아지는 것을 의미한다. 수식으로 나타내면 다음과 같다.

$$
c_1 x_1 + c_2 x_2 + c_3 x_3 + \cdots + c_J x_J = 0
$$

이 수식에서 $c_j$는 스칼라 상수고 $x_j$는 독립변수 벡터다.

다음은 실무에서 발생하기 쉬운 선형조합의 특수한 경우다.

1) 두 독립변수 벡터가 단위만 다르고 실질적으로 같은 경우. 예를 들어 독립변수 데이터 집합 안에 같은 열이 있거나 같은 값인데 단위만 다른 데이터 열이 같이 있는 경우

$$
x_1 - c x_2 = 0 \;\; \leftrightarrow \;\; x_1 = c x_2
$$

2) 몇개의 독립변수를 합하거나 평균하면 다른 독립변수가 되는 경우. 예를 들어 독립변수 데이터 집합 안에 각 과목의 점수와 모든 과목의 총점이 같이 있는 경우  

$$
x_1 + x_2 + x_3 - x_4 = 0 \;\; \leftrightarrow \;\; x_4 = x_1 + x_2 + x_3
$$

이 경우에는 $X^TX$ 행렬의 역행렬이 존재하지 않으므로 수학적으로는 선형회귀분석의 가중치가 존재하지 않는다.

실제로 보스턴 집값 데이터를 사용하여 위와 같은 경우를 실행해보자. 

In [21]:
boston_case1 = boston.copy()
boston_case1["nox10"] = boston_case1.nox * 10

result_case1 = sm.OLS.from_formula("medv ~ nox + nox10", boston_case1).fit()
print(result_case1.summary())

                            OLS Regression Results                            
Dep. Variable:                   medv   R-squared:                       0.183
Model:                            OLS   Adj. R-squared:                  0.181
Method:                 Least Squares   F-statistic:                     112.6
Date:                Fri, 12 Aug 2022   Prob (F-statistic):           7.07e-24
Time:                        18:00:43   Log-Likelihood:                -1789.2
No. Observations:                 506   AIC:                             3582.
Df Residuals:                     504   BIC:                             3591.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     41.3459      1.811     22.828      0.0

In [22]:
boston_case2 = boston.copy()
boston_case2["nox_tax"] = boston_case2.nox + boston_case2.tax

result_case2 = sm.OLS.from_formula("medv ~ nox + tax + nox_tax", boston_case2).fit()
print(result_case2.summary())

                            OLS Regression Results                            
Dep. Variable:                   medv   R-squared:                       0.243
Model:                            OLS   Adj. R-squared:                  0.240
Method:                 Least Squares   F-statistic:                     80.79
Date:                Fri, 12 Aug 2022   Prob (F-statistic):           3.75e-31
Time:                        18:00:43   Log-Likelihood:                -1769.8
No. Observations:                 506   AIC:                             3546.
Df Residuals:                     503   BIC:                             3558.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     38.9878      1.784     21.857      0.0

실행이 되어 가중치가 계산되어 나왔지만 조건수가 아주 커지고 다음과 같은 주의사항이 출력되었다.

```
[2] The smallest eigenvalue is xxxx. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
```

해석하면 다음과 같다.

```
[2] 가장 작은 고유값이 xxxx이다. 강한 다중공선성이 있거나 실험행렬이 특이행렬(singular matrix)이다.
```

특이행렬(singular matrix)은 역행렬이 존재하지 않는 행렬을 말한다. 이 경우 수학적으로는 원래 가중치 계산 자체가 되면 안되지만 현실에서는 미세한 수치적 차이로 인해 대부분의 경우 가중치가 계산되어 나오는 경우가 많다.